In [1]:
from xgboost.sklearn import XGBRegressor

/opt/anaconda3/envs/cs2020/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [25]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as stats
from collections import defaultdict
from itertools import product
from sklearn.metrics import mean_absolute_error as mae

In [3]:
dat = pd.read_csv('train.csv', index_col='row_id')

In [27]:
for x, y in kf.split(dat):
    print(x)

[169767 169768 169769 ... 848832 848833 848834]
[     0      1      2 ... 848832 848833 848834]
[     0      1      2 ... 848832 848833 848834]
[     0      1      2 ... 848832 848833 848834]
[     0      1      2 ... 679065 679066 679067]


the test data is on a Monday afternoon, produce two test sets in Monday after for checking

In [4]:
test_periods = [
    ['1991-08-26 12:00:00', '1991-09-02 24:00:00'],
    ['1991-09-02 12:00:00', '1991-09-02 24:00:00'],
    ['1991-09-09 12:00:00', '1991-09-09 24:00:00'],
    ['1991-09-16 12:00:00', '1991-09-16 24:00:00']]

In [5]:
def preprocess(dat):
    
    time_mapper = {}
    ii = 0
    for h in range(24):
        for mm in ['00','20','40']:
            hh = '{0:02d}'.format(h)
            time_mapper[hh+':'+mm] = ii
            ii += 1
    
    # define the road/day/time_stamp
    dat['unique'] = dat['x'].astype(str) + dat['y'].astype(str) + dat['direction']
    dat['day'] = pd.to_datetime(dat['time']).dt.weekday
    dat['time_stamp'] = dat['time'].apply(lambda x:time_mapper[x.split()[1][:5]])
    uniques = dat['unique'].unique()

    # define the medians
    tmp = dat.groupby(['unique','day','time_stamp']).agg({'congestion':np.median})
    median_mapper = tmp.to_dict()['congestion']
    
    # define the modes
    tmp = dat.groupby(['unique','day','time_stamp']).\
                    agg({'congestion': lambda x: stats.mode(x).mode[0]})
    mode_mapper = tmp.to_dict()['congestion']
    
    # define the diff
    diff_mapper = {}
    for unique in uniques:
        tmp = dat.loc[dat['unique']==unique,['day','time_stamp','congestion']]
        tmp['diff'] = tmp['congestion'].diff()
        tmp = tmp.groupby(['day','time_stamp']).agg({'diff':lambda x: sum(x==0)/len(x)})
        for idx in tmp.index:
            diff_mapper[(unique, idx[0], idx[1])] = tmp.at[idx,'diff']
    
    # define the total
    sums = dat.groupby('time').agg({'congestion':sum})
    
    congestions = dat[['unique','congestion','time']].set_index('time')
    congestions = congestions.groupby(['unique','time']).mean().unstack('unique')
    congestions.columns = congestions.columns.droplevel(0)
    
    cols = congestions.columns.to_list()
    for col in cols:
        col_new = col + '-1'
        congestions[col_new] = np.hstack((0, congestions[col][:-1]))
        col_new = col + '-2'
        congestions[col_new] = np.hstack((0, 0, congestions[col][:-2]))
    congestions = congestions.copy()
    congestions = congestions.drop(index=congestions.index[:2], axis=1)
    
    sums.columns = ['total']
    sums['total-1'] = np.hstack((0, sums['total'][:-1]))
    sums['total-2'] = np.hstack((0, 0, sums['total'][:-2]))
    sums = sums.drop(index=sums.index[:2], axis=1)
    
    congestions['time_stamp'] = \
       pd.Series(congestions.index).apply(lambda x:time_mapper[x.split()[1][:5]]).values
    congestions['day'] = \
       pd.to_datetime(pd.Series(congestions.index)).dt.weekday.values
    
    congestions = pd.merge(congestions, sums, left_on='time', right_on='time', how='outer')
    
    return congestions, uniques, median_mapper, time_mapper, diff_mapper, mode_mapper

In [6]:
congestions, uniques, median_mapper, time_mapper, diff_mapper, mode_mapper = preprocess(dat)

/var/folders/h9/t5gv56tx3wl0wk4czyrvf36m0000gn/T/ipykernel_79581/2627335551.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  congestions[col_new] = np.hstack((0, 0, congestions[col][:-2]))
/var/folders/h9/t5gv56tx3wl0wk4czyrvf36m0000gn/T/ipykernel_79581/2627335551.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  congestions[col_new] = np.hstack((0, congestions[col][:-1]))
/var/folders/h9/t5gv56tx3wl0wk4czyrvf36m0000gn/T/ipykernel_79581/2627335551.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually

/var/folders/h9/t5gv56tx3wl0wk4czyrvf36m0000gn/T/ipykernel_79581/2627335551.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  congestions[col_new] = np.hstack((0, 0, congestions[col][:-2]))
/var/folders/h9/t5gv56tx3wl0wk4czyrvf36m0000gn/T/ipykernel_79581/2627335551.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  congestions[col_new] = np.hstack((0, congestions[col][:-1]))
/var/folders/h9/t5gv56tx3wl0wk4czyrvf36m0000gn/T/ipykernel_79581/2627335551.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually

In [7]:
congestions

,00EB,00NB,00SB,01EB,01NB,01SB,01WB,02EB,02NB,02SB,...,23SB-2,23SW-1,23SW-2,23WB-1,23WB-2,time_stamp,day,total,total-1,total-2
time,,,,,,,,,,,,,,,,,,,,,
1991-04-01 00:40:00,70.0,49.0,24.0,31.0,51.0,57.0,91.0,42.0,41.0,35.0,...,70.0,29.0,29.0,57.0,26.0,2,0,2735,2730,2691
1991-04-01 01:00:00,70.0,49.0,24.0,36.0,51.0,46.0,49.0,39.0,44.0,58.0,...,80.0,29.0,29.0,48.0,57.0,3,0,2692,2735,2730
1991-04-01 01:20:00,70.0,49.0,24.0,29.0,53.0,83.0,62.0,36.0,46.0,46.0,...,79.0,29.0,29.0,44.0,48.0,4,0,2658,2692,2735
1991-04-01 01:40:00,70.0,49.0,24.0,15.0,48.0,38.0,39.0,36.0,51.0,43.0,...,77.0,29.0,29.0,44.0,44.0,5,0,2668,2658,2692
1991-04-01 02:00:00,70.0,49.0,65.0,20.0,27.0,46.0,39.0,35.0,56.0,49.0,...,79.0,29.0,29.0,44.0,44.0,6,0,2904,2668,2658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991-09-30 10:20:00,43.0,29.0,60.0,23.0,63.0,61.0,42.0,47.0,51.0,56.0,...,66.0,24.0,35.0,33.0,41.0,31,0,3058,2977,3008
1991-09-30 10:40:00,47.0,28.0,35.0,24.0,62.0,63.0,32.0,44.0,57.0,65.0,...,69.0,22.0,24.0,49.0,33.0,32,0,2922,3058,2977
1991-09-30 11:00:00,40.0,31.0,53.0,22.0,65.0,58.0,38.0,44.0,60.0,62.0,...,68.0,20.0,22.0,47.0,49.0,33,0,2952,2922,3058


In [8]:
def calconnect():
    connected = defaultdict(list)
    tmp ={'E':['0','1','2'],
          'N':['0','1','2','3'],
          'W':['0','1','2'],
          'S':['0','1','2','3']}
    diag = []
    for unique in uniques:
        if unique[3] != 'B': 
            diag.append(unique+'-1')
            diag.append(unique+'-2')
            
    for unique in uniques:
        if unique[3] != 'B':  
            connected[unique] = diag[:]
            continue
            
        if unique[2] == 'E' or unique[2] == 'W':
            y = unique[1]
            for x in tmp[unique[2]]:
                new = x + y + 'EB'
                if new in uniques: 
                    connected[unique].append(new+'-1')
                    connected[unique].append(new+'-2')
                new = x + y + 'WB'
                if new in uniques: 
                    connected[unique].append(new+'-1')
                    connected[unique].append(new+'-2')
        else:
            x = unique[0]
            for y in tmp[unique[2]]:
                new = x + y + 'NB'
                if new in uniques: 
                    connected[unique].append(new+'-1')
                    connected[unique].append(new+'-2')
                new = x + y + 'SB'
                if new in uniques: 
                    connected[unique].append(new+'-1')
                    connected[unique].append(new+'-2')
            
    return connected

In [9]:
connected = calconnect()

In [10]:
connected

defaultdict(list,
            {'00EB': ['00EB-1',
              '00EB-2',
              '10EB-1',
              '10EB-2',
              '10WB-1',
              '10WB-2',
              '20EB-1',
              '20EB-2',
              '20WB-1',
              '20WB-2'],
             '00NB': ['00NB-1',
              '00NB-2',
              '00SB-1',
              '00SB-2',
              '01NB-1',
              '01NB-2',
              '01SB-1',
              '01SB-2',
              '02NB-1',
              '02NB-2',
              '02SB-1',
              '02SB-2',
              '03NB-1',
              '03NB-2',
              '03SB-1',
              '03SB-2'],
             '00SB': ['00NB-1',
              '00NB-2',
              '00SB-1',
              '00SB-2',
              '01NB-1',
              '01NB-2',
              '01SB-1',
              '01SB-2',
              '02NB-1',
              '02NB-2',
              '02SB-1',
              '02SB-2',
              '03NB-1',
              '03NB-

In [11]:
def assemble(col):
    cols = connected[col][:]
    #cols = [col for col in congestions.columns.to_list() if '-' in col]
    cols.append('time_stamp')
    cols.append('day')
    cols.append('total-1')
    cols.append('total-2')
    dat = congestions[cols].copy()  #the copy is needed
    dat['median'] = dat.apply(\
                        lambda x: median_mapper[(col, x['day'].astype(int), 
                                                 x['time_stamp'].astype(int))], axis=1).values
    dat['diff'] = dat.apply(\
                        lambda x: diff_mapper[(col, x['day'].astype(int), 
                                               x['time_stamp'].astype(int))], axis=1).values
    dat['mode'] = dat.apply(\
                        lambda x: mode_mapper[(col, x['day'].astype(int), 
                                               x['time_stamp'].astype(int))], axis=1).values
    dat['target'] = congestions[col].values
    
    trains, tests = [], []
    for period in test_periods:
        trains.append(dat[dat.index < period[0]])
        tests.append(dat[(dat.index >= period[0]) & (dat.index <= period[1])])
    return trains, tests, dat

In [13]:
col = '22SE'
trains, tests, train_all = assemble(col)

In [14]:
trains[0]

,03NE-1,03NE-2,03SW-1,03SW-2,10NE-1,10NE-2,10SW-1,10SW-2,12NE-1,12NE-2,...,23SW-1,23SW-2,time_stamp,day,total-1,total-2,median,diff,mode,target
time,,,,,,,,,,,,,,,,,,,,,
1991-04-01 00:40:00,17.0,21.0,48.0,47.0,43.0,43.0,52.0,52.0,24.0,24.0,...,29.0,29.0,2,0,2730,2691,40.0,0.555556,40,51.0
1991-04-01 01:00:00,46.0,17.0,15.0,48.0,43.0,43.0,52.0,52.0,24.0,24.0,...,29.0,29.0,3,0,2735,2730,40.0,0.962963,40,51.0
1991-04-01 01:20:00,50.0,46.0,31.0,15.0,43.0,43.0,52.0,52.0,24.0,24.0,...,29.0,29.0,4,0,2692,2735,40.0,1.000000,40,51.0
1991-04-01 01:40:00,50.0,50.0,19.0,31.0,43.0,43.0,52.0,52.0,24.0,24.0,...,29.0,29.0,5,0,2658,2692,40.0,1.000000,40,51.0
1991-04-01 02:00:00,48.0,50.0,26.0,19.0,43.0,43.0,52.0,52.0,24.0,24.0,...,29.0,29.0,6,0,2668,2658,40.0,0.814815,40,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991-08-26 10:20:00,28.0,32.0,28.0,29.0,65.0,63.0,45.0,34.0,48.0,38.0,...,19.0,20.0,31,0,3190,3180,20.0,0.148148,20,20.0
1991-08-26 10:40:00,24.0,28.0,23.0,28.0,72.0,65.0,43.0,45.0,49.0,48.0,...,20.0,19.0,32,0,3049,3190,20.0,0.518519,20,20.0
1991-08-26 11:00:00,30.0,24.0,31.0,23.0,61.0,72.0,45.0,43.0,55.0,49.0,...,21.0,20.0,33,0,3090,3049,20.0,0.592593,20,21.0


In [12]:
def gridsearch_xgboost(params, trains, tests, obj="reg:pseudohubererror"):
    ''' the params include both the fit and the booster params '''
    
    param_names, param_vals = zip(*list(params.items()))
    id_booster, id_fit = [], []
    for idx, name in enumerate(param_names):
        if (name=='early_stopping_rounds'):
            id_fit.append(idx)
        else:
            id_booster.append(idx)
    
    best_test_score = 999.0
    for val_combination in product(*param_vals):
        param_fit, param_booster = {}, {}
        param_fit['eval_metric'] = 'mae'
        for idx in id_fit:
            param_fit[param_names[idx]] = val_combination[idx]
        for idx in id_booster:
            param_booster[param_names[idx]] = val_combination[idx]
        print(f"param_fit: {param_fit}")
        print(f"param_booster: {param_booster}")
        
        regr = XGBRegressor(**param_booster, objective=obj)
        result = defaultdict(list)
        for train, test in zip(trains,tests):
            X_train, y_train = train.drop(columns='target').values, train['target'].values
            X_test, y_test = test.drop(columns='target').values, test['target'].values
            param_fit['eval_set'] = [(X_test, y_test)]            
            regr.fit(X_train, y_train, **param_fit, verbose=False)
            if hasattr(regr, 'best_ntree_limit'):
                result['ntree'].append(regr.best_ntree_limit)
            else:
                result['ntree'].append(regr.n_estimators)            
            result['score'].append(mae(regr.predict(X_test),y_test))
            print(f'stop at {result["ntree"][-1]}, score={result["score"][-1]}')

        mean_test_score = np.mean(result['score'])
        print(f"mean_test_score={mean_test_score}")
        if mean_test_score < best_test_score:
            best_test_score = mean_test_score
            best_param_booster = param_booster
            best_param_booster['n_estimators'] = np.int32(np.mean(result['ntree']))
    
    return best_param_booster

In [ ]:
trains[0].head()

In [ ]:
tests[0].head()

In [ ]:
params = {'gamma': [0, 1.0],
          'max_depth': [6],
          'min_child_weight': [0.0, 1.0, 2.0],
          'n_estimators': [10000],
          'early_stopping_rounds': [50],
          'learning_rate': [1e-3, 1e-2]}

In [ ]:
col = '00EB'
trains, tests, train_all = assemble(col)
best_param_booster = gridsearch_xgboost(params, trains, tests)

In [ ]:
best_param_booster

In [19]:
params = {'gamma': [0.0],
          'max_depth': [6],
          'min_child_weight': [0.0],
          'n_estimators': [10000],
          'early_stopping_rounds': [50],
          'learning_rate': [1e-3,1e-2]}

In [22]:
regrs = {}

for col in uniques[5:6]:
    trains, tests, train_all = assemble(col)
    
    best_param_booster = gridsearch_xgboost(params, trains, tests)
    
    print(f"refitting with best param={best_param_booster}")
    regr = XGBRegressor(**best_param_booster, objective="reg:pseudohubererror")
    X, y = train_all.drop(columns='target').values, train_all['target'].values    
    regr.fit(X,y)
    print(f'mae on train set:{mae(regr.predict(X),y)}')
    regrs[col] = regr

param_fit: {'eval_metric': 'mae', 'early_stopping_rounds': 50}
param_booster: {'gamma': 0.0, 'max_depth': 6, 'min_child_weight': 0.0, 'n_estimators': 10000, 'learning_rate': 0.001}
stop at 4932, score=4.64182383026007
stop at 880, score=3.831267992655436
stop at 3390, score=3.06270334455702
stop at 3509, score=3.566393004523383
mean_test_score=3.7755470429989773
param_fit: {'eval_metric': 'mae', 'early_stopping_rounds': 50}
param_booster: {'gamma': 0.0, 'max_depth': 6, 'min_child_weight': 0.0, 'n_estimators': 10000, 'learning_rate': 0.01}
stop at 905, score=5.063264056303655
stop at 113, score=3.649509006076389
stop at 31, score=44.30111863878038
stop at 1, score=48.48764122856988
mean_test_score=25.375383232432576
refitting with best param={'gamma': 0.0, 'max_depth': 6, 'min_child_weight': 0.0, 'n_estimators': 3177, 'learning_rate': 0.001}
mae on train set:4.003918095004589


In [ ]:
regr

In [ ]:
import pickle

In [ ]:
with open('baseline2_models.pickle', 'wb') as fid:
    pickle.dump(regrs, fid)

In [ ]:
regrs = {}

for col in uniques:
# for col in ['22SE']:
    trains, tests, train_all = assemble(col)
    
    best_mae = 1000.0
    print(f"fitting {col}")
    for leaf in [3,5,10,20,50,100,200]:
        regr = RandomForestRegressor(n_estimators=100, min_samples_leaf=leaf, criterion='absolute_error', n_jobs=48)
        
        results = []
        for train, test in zip(trains, tests):
            X, y = train.drop(columns='target').values, train['target'].values
            regr.fit(X, y)
            
            X, y_test = test.drop(columns='target').values, test['target'].values
            y_pred = regr.predict(X)
            
            results.append(mae(y_pred, y_test))
        
        mean_mae = np.mean(results)
        print(f"{col} leaf {leaf}: {mean_mae}")
        if mean_mae < best_mae:
            best_leaf = leaf
            best_mae = mean_mae
    
    print(f"refitting with best leaf={best_leaf}")
    regr = RandomForestRegressor(n_estimators=100, min_samples_leaf=best_leaf, criterion='absolute_error', n_jobs=48)
    X, y = train_all.drop(columns='target').values, train_all['target'].values
    regr.fit(X,y)
    regrs[col] = regr

In [ ]:
with open('baseline2_models_forcompare.pickle', 'wb') as fid:
    pickle.dump(regrs, fid)

In [ ]:
with open('baseline2_models.pickle','rb') as fid:
    regrs = pickle.load(fid)

In [ ]:
test = pd.read_csv('test.csv')
test.head()

In [ ]:
def preprocess_test(dat, time_mapper):
    dat['unique'] = dat['x'].astype(str) + dat['y'].astype(str) + dat['direction']
    dat['day'] = pd.to_datetime(dat['time']).dt.weekday
    dat['time_stamp'] = dat['time'].apply(lambda x:time_mapper[x.split()[1][:5]])
    congestions = dat[['unique','time']].set_index('time')
    congestions['congestion'] = 0.0
    congestions = congestions.groupby(['unique','time']).mean().unstack('unique')
    congestions.columns = congestions.columns.droplevel(0)
    
    for col in congestions.columns.to_list():
        col_new = col + '-1'
        congestions[col_new] = 0.0
    congestions = congestions.copy()
    
    congestions['time_stamp'] = \
       pd.Series(congestions.index).apply(lambda x:time_mapper[x.split()[1][:5]]).values
    congestions['day'] = \
       pd.to_datetime(pd.Series(congestions.index)).dt.weekday.values
    congestions['total-1'] = 0.0
    
    return congestions

In [ ]:
congestions_test = preprocess_test(test, time_mapper)
congestions_test

In [ ]:
congestions.tail(1)

In [ ]:
col_mapper = {}
cols_tosum = []
for col in uniques:
    col_mapper[col+'-1'] = col
    cols_tosum.append(col+'-1')

In [ ]:
idx_test = congestions_test.index[0]
idx_train = congestions.index[-1]
for col in uniques:
    col1 = col+'-1'
    congestions_test.at[idx_test, col1] = congestions.at[idx_train, col_mapper[col1]]
congestions_test.at[idx_test, 'total-1'] = congestions_test.loc[idx_test, cols_tosum].sum()
congestions_test

In [ ]:
train_all.tail(1)

In [ ]:
connected['00EB']

In [ ]:
def predict():
    
    for i in range(len(congestions_test.index)):
        
        # fill the new row
        idx = congestions_test.index[i]
        if (i>0):
            idx1 = congestions_test.index[i-1]
            for col in uniques:
                congestions_test.at[idx, col+'-1'] = congestions_test.at[idx1, col]
            congestions_test.at[idx, 'total-1'] = congestions_test.loc[idx, cols_tosum].sum()
        
        # make prediction of this row
        for col in uniques:
            cols = connected[col][:]
            cols.append('time_stamp')
            cols.append('day')
            cols.append('total-1')
            dat = congestions_test.loc[[idx], cols]
            dat['median'] = dat.apply(\
                        lambda x: median_mapper[(col, x['day'], x['time_stamp'])], axis=1).values
            dat['diff'] = dat.apply(\
                        lambda x: diff_mapper[(col, x['day'], x['time_stamp'])], axis=1).values
            dat['mode'] = dat.apply(\
                        lambda x: mode_mapper[(col, x['day'], x['time_stamp'])], axis=1).values
            y = regrs[col].predict(dat.values)
            congestions_test.at[idx, col] = y

In [ ]:
predict()

In [ ]:
congestions_test.head()

In [ ]:
congestions.tail(1)

In [ ]:
congestions_test[['00EB','00EB-1']]

In [ ]:
test['congestion'] = 0.0
for idx in test.index:
    time, unique = test.at[idx,'time'], test.at[idx,'unique']
    test.at[idx,'congestion'] = congestions_test.at[time,unique]
test.head()

In [ ]:
test[['row_id', 'congestion']].to_csv('submission_baseline2.csv', index=False)

In [ ]:
dat.head()

In [ ]:
lower = dat.groupby(['unique','day','time_stamp'])['congestion'].quantile(0.15).to_dict()
upper = dat.groupby(['unique','day','time_stamp'])['congestion'].quantile(0.7).to_dict()

In [ ]:
lower

In [ ]:
test

In [ ]:
test['lower'] = test.apply(lambda x:\
    lower[(x['unique'], x['day'], x['time_stamp'])], axis=1)
test['upper'] = test.apply(lambda x:\
    upper[(x['unique'], x['day'], x['time_stamp'])], axis=1)
test

In [ ]:
test['congestion_tweak'] = test['congestion'].clip(test['lower'],test['upper'])

In [ ]:
test[['row_id', 'congestion_tweak']].to_csv('submission_baseline1_tweaked.csv', index=False)